In [4]:
import csv
import pandas as pd
from collections import defaultdict

# Helper function to determine the maximum number of columns in a CSV file.
def max_columns_in_csv(filepath):
    with open(filepath, newline='') as f:
        reader = csv.reader(f)
        return max(len(row) for row in reader)

# --- Load CSV files using the maximum number of columns ---

# total_shark_class_sub_parts.csv
max_fields_total = max_columns_in_csv('total_shark_class_sub_parts.csv')
df_total = pd.read_csv('total_shark_class_sub_parts.csv', 
                       header=None, 
                       engine='python', 
                       names=range(max_fields_total))

# recipe_book.csv
max_fields_recipe_book = max_columns_in_csv('recipe_book.csv')
df_recipe_book = pd.read_csv('recipe_book.csv', 
                             header=None, 
                             engine='python', 
                             names=range(max_fields_recipe_book))

# recipe_gathering.csv
max_fields_recipe_gathering = max_columns_in_csv('recipe_gathering.csv')
df_recipe_gathering = pd.read_csv('recipe_gathering.csv', 
                                  header=None, 
                                  engine='python', 
                                  names=range(max_fields_recipe_gathering))

# --- Build a recipe dictionary from recipe_book.csv ---
recipes = {}
for _, row in df_recipe_book.iterrows():
    product = row[0]
    ingredients = []
    # Iterate over columns in steps of two (starting at column 1)
    for i in range(1, max_fields_recipe_book, 2):
        if pd.isna(row[i]):
            break
        ingredient = row[i]
        if i+1 < max_fields_recipe_book and not pd.isna(row[i+1]):
            qty = float(row[i+1])
        else:
            qty = 0
        ingredients.append((ingredient, qty))
    recipes[product] = ingredients

# --- Build a dictionary of top-level items from total_shark_class_sub_parts.csv ---
# The first column is the product and the second column is the required quantity.
top_level = {}
for _, row in df_total.iterrows():
    product = row[0]
    qty = float(row[1])
    top_level[product] = qty

# --- Recursive function to compute base ingredients ---
requirements = defaultdict(float)

def compute_requirements(item, multiplier):
    if item in recipes:
        for ingredient, qty in recipes[item]:
            compute_requirements(ingredient, qty * multiplier)
    else:
        requirements[item] += multiplier

# Process each top-level item.
for product, qty in top_level.items():
    compute_requirements(product, qty)

df_requirements = pd.DataFrame(list(requirements.items()), 
                               columns=["Ingredient", "Total Quantity"])

# --- Process the gathering CSV ---
def combine_location(row):
    parts = [str(x) for x in row[1:] if pd.notna(x)]
    return ', '.join(parts)

df_recipe_gathering["Location Info"] = df_recipe_gathering.apply(combine_location, axis=1)
# Select only the first column (ingredient name) and the combined location info column.
df_recipe_gathering = df_recipe_gathering[[0, "Location Info"]]
df_recipe_gathering.columns = ["Ingredient", "Location Info"]

# --- Merge and output the final CSV ---
df_output = pd.merge(df_requirements, df_recipe_gathering, on="Ingredient", how="left")
df_output.to_csv('gathering_list.csv', index=False)
df_output


,Ingredient,Total Quantity,Location Info
0,Walnut Log,54.0,South Shroud ( 23.1 / 21.6 )
1,Wind Shard,351.0,NaN
2,Spruce Log,126.0,"Coerthas Central Highlands ( 29.5 / 13.1 ), Ti..."
3,Wind Crystal,279.0,NaN
4,Iron Ingot,78.0,NaN
5,Ice/Fire Shard,3000.0,NaN
6,Cobalt Ore,450.0,Northern Thanalan ( 23.1 / 24.2 )
7,Iron Ore,807.0,Western Thanalan ( 27.2 / 17.4 )
8,Cedar Log,45.0,Coerthas Western Highlands ( 31.3 / 32.1 )
9,Mythril Ore,342.0,Southern Thanalan ( 25.2 / 39.9 )
